In [1]:
import time
import numpy as np
import pandas as pd
import torch

import torch.optim as optim

In [2]:
from src.utils_data import M4TS
from src.utils_data import get_m4_all_series

from src.utils_config import ModelConfig
from src.utils_visualization import plot_prediction

from src.utils_train import PinballLoss, LevelVAriabilityLoss
# from src.utils_models import VanillaLSTM
# from src.utils_models import ES
from src.utils_models import ES, RNN, ESRNN

In [3]:
mc = ModelConfig(config_file='configs/config_m4quarterly.yaml',
                 root_dir='./results/')

all_series = get_m4_all_series(mc=mc, data='train')
# train(mc, all_series)

# model = ESRNN(mc)
# print("\n Model parameters: \n", model.rnn.parameters, "\n")

In [4]:
def train(mc, all_series):
    print(10*'='+' Training {} '.format(mc.dataset_name) + 10*'=')
    
    # Random Seeds
    torch.manual_seed(mc.copy)
    np.random.seed(mc.copy)

    # Model
    esrnn = ESRNN(mc)

    # Optimizers
    es_optimizer = optim.Adam(params=esrnn.es.parameters(),
                              lr=mc.learning_rate*mc.per_series_lr_multip, 
                              betas=(0.9, 0.999), eps=mc.gradient_eps)

    rnn_optimizer = optim.Adam(params=esrnn.rnn.parameters(),
                         lr=mc.learning_rate,
                         betas=(0.9, 0.999), eps=mc.gradient_eps)
    
    # Loss Functions
    pinball_loss = PinballLoss(mc.tau)
    level_variability_loss = LevelVAriabilityLoss(mc.level_variability_penalty)

    # training code
    for epoch in range(mc.max_epochs):
        start = time.time()
        forecast_losses = []
        lev_variability_losses = []
        state_losses = []
        
        for j in range(len(all_series)):
            ts_object = all_series[j]
            y = ts_object.y
            idxs = ts_object.idxs
            n_series, n_time = y.shape
            
            levels_ex, seasonalities_ex, log_diff_of_levels = esrnn.compute_levels_seasons(ts_object)
            
            losses = []
            for i in range(mc.input_size-1, n_time-mc.output_size):
                input_start = i+1-mc.input_size
                input_end = i+1
                
                # Deseasonalization and normalization
                input_ex = y[:, input_start:input_end] / seasonalities_ex[:, input_start:input_end]
                input_ex = input_ex / levels_ex[:, i]
                #input_ex = dy.noise(dy.log(input_ex), mc.noise_std)
                input_ex = torch.log(input_ex)
                
                # Concatenate categories
                categories_ex = ts_object.categories_vect
                input_ex = torch.cat((input_ex, categories_ex), 1)
                
                print("input_ex.shape", input_ex.shape)
                print("y[:, input_start:input_end].shape", y[:, input_start:input_end].shape)
                print("seasonalities_ex[:, input_start:input_end].shape", \
                      seasonalities_ex[:, input_start:input_end].shape)
                print("levels_ex[:, i].shape", levels_ex[:, i].shape)
                
                output_ex = esrnn.rnn(input_ex)
                
                labels_start = i+1
                labels_end = i+1+mc.output_size
                
                # Deseasonalization and normalization
                labels_ex = y[:, labels_start:labels_end] / seasonalities_ex[labels_start:labels_end]
                labels_ex = torch.log(labels_ex) / levels_ex[:, i]
                
                loss_ex = pinball_loss(labels_ex, output_ex)
                losses.append(loss_ex)
            
            # Losses
            forecloss_ex = dy.average(losses)
            loss_ex = forecloss_ex
            forecast_losses.append(forecloss_ex.npvalue())

            if mc.level_variability_penalty>0:
                level_var_loss_ex = level_variability_loss(log_diff_of_levels, 
                                                        mc.level_variability_penalty)
                loss_ex += level_var_loss_ex
                lev_variability_losses.append(level_var_loss_ex.npvalue())
            
            loss_ex.backward()
            rnn_optimizer.step()
            es_optimizer.step()
        
        if epoch >= (mc.max_epochs-mc.averaging_level):
            copy = epoch+mc.averaging_level-mc.max_epochs
            esrnn.save(copy=copy)

        print("========= Epoch {} finished =========".format(epoch))
        print("Training time: {}".format(time.time()-start))
        print("Forecast loss: {}".format(np.mean(forecloss_ex.npvalue())))

    print('Train finished!')

In [5]:
train(mc, all_series)

========== Training m4quarterly ==========
input_ex.shape torch.Size([1, 10])
y[:, input_start:input_end].shape torch.Size([1, 4])
seasonalities_ex[:, input_start:input_end].shape torch.Size([1, 4])
levels_ex[:, i].shape torch.Size([1])
levels_ex[:, i].shape torch.Size([1])


IndexError: too many indices for tensor of dimension 2

In [ ]:
model = RNN(mc)
print("\n Model parameters: \n", model.rnn.parameters, "\n")

In [ ]:
np.float32(ver)